In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

In [2]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

In [3]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

# fume_query(target="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", server="biotech_main", start="2021-12-25T00:00:00.000", end=current_date())
# fume_query(target="#biotech_weather/temperature_tn", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

In [4]:
# Arguments: Sash Point, Occ Point, Server Name, Start Time, End Time
# Returns: Total time that hood sash was open when room is unoccupied, aggregated by hour

def total_time_sash_open_unoccupied(sash_point, occ_point, server, start, end):
  sash_master = fume_query(sash_point, server, start, end)
  occ_master = fume_query(occ_point, server, start, end)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  # sash_list.to_csv("test.csv")
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)

  df = pd.concat([sash_list, occ_list], axis=1)
  df.columns = ["sash", "occ"]
  display(df)

  time_interval = df.index[1].minute - df.index[0].minute

  # 1.2 inches is arbitrary
  df["time_open_bool"] = (df["sash"] > 1.2) & (df["occ"] == 0)
  df["time_open_mins"] = np.where(df["time_open_bool"] == False, 0, time_interval)

  df = df.dropna()
  display(df)
  # df.to_csv("test2.csv")

  df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  return df["time_open_mins"]

total_time_sash_open_unoccupied(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

<Response [200]>
<Response [200]>

Sash List:  2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

Sash List new:  2021-11-17 05:00:00    3.3
2021-11-17 05:05:00    3.3
2021-11-17 05:10:00    3.3
2021-11-17 05:15:00    3.3
2021-11-17 05:20:00    3.3
                      ... 
2021-11-18 04:35:00    1.8
2021-11-18 04:40:00    1.8
2021-11-18 04:45:00    1.8
2021-11-18 04:50:00    1.8
2021-11-18 04:55:00    1.8
Length: 288, dtype: float64

OCC List:  2021-11-17 05:00:00    1.0
2021-11-17 05:05:00    1.0
2021-11-17 05:10:00    1.0
2021-11-17 05:15:00    1.0
2021-11-17 05:20:00    1.0
                      ... 
2021-11-18 04:35:00    0.0
2021-11-18 04:40:00    0.0
2021-11-18 04:45:00    0.0
2021-11-18 04:50:00    0.0
2021

,sash,occ
2021-11-17 05:00:00,3.3,1.0
2021-11-17 05:05:00,3.3,1.0
2021-11-17 05:10:00,3.3,1.0
2021-11-17 05:15:00,3.3,1.0
2021-11-17 05:20:00,3.3,1.0
...,...,...
2021-11-18 04:35:00,1.8,0.0
2021-11-18 04:40:00,1.8,0.0
2021-11-18 04:45:00,1.8,0.0
2021-11-18 04:50:00,1.8,0.0


,sash,occ,time_open_bool,time_open_mins
2021-11-17 05:00:00,3.3,1.0,False,0
2021-11-17 05:05:00,3.3,1.0,False,0
2021-11-17 05:10:00,3.3,1.0,False,0
2021-11-17 05:15:00,3.3,1.0,False,0
2021-11-17 05:20:00,3.3,1.0,False,0
...,...,...,...,...
2021-11-18 04:35:00,1.8,0.0,True,5
2021-11-18 04:40:00,1.8,0.0,True,5
2021-11-18 04:45:00,1.8,0.0,True,5
2021-11-18 04:50:00,1.8,0.0,True,5


2021-11-17 06:00:00     0
2021-11-17 07:00:00    10
2021-11-17 08:00:00    50
2021-11-17 09:00:00     0
2021-11-17 10:00:00    30
2021-11-17 11:00:00    40
2021-11-17 12:00:00    15
2021-11-17 13:00:00    40
2021-11-17 14:00:00    10
2021-11-17 15:00:00     0
2021-11-17 16:00:00     0
2021-11-17 17:00:00     0
2021-11-17 18:00:00     0
2021-11-17 19:00:00     0
2021-11-17 20:00:00     0
2021-11-17 21:00:00     0
2021-11-17 22:00:00     0
2021-11-17 23:00:00     0
2021-11-18 00:00:00     0
2021-11-18 01:00:00     0
2021-11-18 02:00:00     0
2021-11-18 03:00:00    55
2021-11-18 04:00:00    25
2021-11-18 05:00:00    30
Freq: 60T, Name: time_open_mins, dtype: int64

In [5]:
# Arguments: Sash Point, Occ Point, Server Name, Start Time, End Time
# Returns: Total time that hood sash was open when room is unoccupied, aggregated by hour

# A switch in the energy calculations that allows us to avoid issues between times of years where it is hotter/colder outside.
def coldorhot(cfm,external,internal):
    if external<=internal:
        #sensible heating equation
        return 1.08 * cfm * (internal - external)
    if external>internal:
        #enthalpy of air
        return 0.24 * cfm /13.333 * 60 * (external - internal)

def total_energy(cfm_point, sash_point, occ_point, internal_temp_point, external_temp_point, server, start, end):
  #external_temp_master = outside_temp(start,end)
  cfm_master = fume_query(cfm_point,server,start,end)
  sash_master = fume_query(sash_point,server,start,end)
  occ_master = fume_query(occ_point,server,start,end)
  internal_temp_master = fume_query(internal_temp_point,server,start,end)
  external_temp_master = fume_query(external_temp_point, server, start, end)
  print("\nExternal Temp Master ", external_temp_master)
  
  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List new: ", external_temp_list)
  print("\nLength of External Temp List new: ", len(external_temp_list))

  # interpolating External Temperature values
  ext_temp_hr = external_temp_list.tolist()
  ext_temp = []
  i = 0
  while i < (len(ext_temp_hr)-1):
    initial = ext_temp_hr[i]
    difference = (ext_temp_hr[i+1] - initial)/3
    ext_temp_5 = []
    for x in range(3):
      ext_temp_5.append(initial + difference*x)
    print("\n 3 Ext Temp Values: ", ext_temp_5)
    ext_temp = ext_temp + ext_temp_5
    i += 1
  ext_temp = ext_temp[3:] + [ext_temp_hr[-1]]
  print("\nList of New Ext Temp Values: ", ext_temp)

  # Internal Temperatures span multiple days, External Temperatures span one day
  new_external_temp_list = pd.Series(data=ext_temp, index=[i[1] for i in internal_temp_master[:len(ext_temp)]])
  print("\nNew External Temp List: ", new_external_temp_list)

  new_df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, new_external_temp_list], axis=1)
  new_df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(new_df)
  new_df = new_df.dropna()
  
  new_df['BTUh'] = new_df.apply(lambda new_df: coldorhot(new_df['cfm'], new_df['external temp'], new_df['internal temp']), axis=1)
  print("\nFinal Data Frame: ")
  display(new_df)

  # NEED TO MAKE HOURLY BTUH!!!!

  new_df = new_df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  return new_df["BTUh"]

total_energy(cfm_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log", sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend", internal_temp_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log", external_temp_point="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat", server = "biotech_main", start=str(datetime(2021, 11, 17)), end=str(datetime(2021, 11, 18)))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>

External Temp Master  [(35.2, '2021-11-17T05:00:00.000') (35.3, '2021-11-17T05:15:00.000')
 (35.5, '2021-11-17T05:30:00.000') (35.6, '2021-11-17T05:45:00.000')
 (35.3, '2021-11-17T06:00:00.000') (35.1, '2021-11-17T06:15:00.000')
 (35.3, '2021-11-17T06:30:00.000') (35.7, '2021-11-17T06:45:00.000')
 (35.8, '2021-11-17T07:00:00.000') (35.5, '2021-11-17T07:15:00.000')
 (35.1, '2021-11-17T07:30:00.000') (34.5, '2021-11-17T07:45:00.000')
 (34.5, '2021-11-17T08:00:00.000') (34.7, '2021-11-17T08:15:00.000')
 (35.6, '2021-11-17T08:30:00.000') (35.7, '2021-11-17T08:45:00.000')
 (35.5, '2021-11-17T09:00:00.000') (34.3, '2021-11-17T09:15:00.000')
 (33.8, '2021-11-17T09:30:00.000') (34.5, '2021-11-17T09:45:00.000')
 (35.7, '2021-11-17T10:00:00.000') (35.5, '2021-11-17T10:15:00.000')
 (36.1, '2021-11-17T10:30:00.000') (36. , '2021-11-17T10:45:00.000')
 (37.3, '2021-11-17T11:00:00.000') (38.1, '2021-11-17T11:15:00.0

,cfm,sash,occ,internal temp,external temp
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.300000
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,35.366667
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,35.433333
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.500000
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,35.533333
...,...,...,...,...,...
2021-11-19 04:35:00,222.66,2.4,0.0,70.3,NaN
2021-11-19 04:40:00,224.13,2.4,0.0,70.3,NaN
2021-11-19 04:45:00,225.10,2.4,0.0,70.3,NaN
2021-11-19 04:50:00,223.64,2.4,0.0,70.3,NaN



Final Data Frame: 


,cfm,sash,occ,internal temp,external temp,BTUh
2021-11-17 05:00:00,224.62,3.3,1.0,72.2,35.300000,8951.55624
2021-11-17 05:05:00,223.15,3.3,1.0,72.2,35.366667,8876.90700
2021-11-17 05:10:00,224.13,3.3,1.0,72.2,35.433333,8899.75404
2021-11-17 05:15:00,224.13,3.3,1.0,72.2,35.500000,8883.61668
2021-11-17 05:20:00,224.62,3.3,1.0,72.2,35.533333,8894.95200
...,...,...,...,...,...,...
2021-11-19 04:10:00,223.15,2.4,0.0,70.4,43.033333,6595.42140
2021-11-19 04:15:00,222.66,2.4,0.0,70.4,43.000000,6588.95472
2021-11-19 04:20:00,227.06,2.4,0.0,70.4,42.900000,6743.68200
2021-11-19 04:25:00,224.62,2.4,0.0,70.3,42.800000,6671.21400


2021-11-17 06:00:00    106772.80824
2021-11-17 07:00:00    106397.59500
2021-11-17 08:00:00    106844.58972
2021-11-17 09:00:00    105010.18920
2021-11-17 10:00:00    107453.12220
2021-11-17 11:00:00    101129.95080
2021-11-17 12:00:00     96860.20572
2021-11-17 13:00:00     95726.50164
2021-11-17 14:00:00     90536.34240
2021-11-17 15:00:00     86548.72212
2021-11-17 16:00:00     84739.81788
2021-11-17 17:00:00     80273.53044
2021-11-17 18:00:00     75026.76228
2021-11-17 19:00:00     71093.90016
2021-11-17 20:00:00     67903.34616
2021-11-17 21:00:00     90693.02232
2021-11-17 22:00:00     89272.08540
2021-11-17 23:00:00     89160.51924
2021-11-18 00:00:00     88235.87076
2021-11-18 01:00:00     86701.26096
2021-11-18 02:00:00     69889.84272
2021-11-18 03:00:00     61407.02304
2021-11-18 04:00:00     61207.65432
2021-11-18 05:00:00     60750.00288
2021-11-18 06:00:00     59011.60860
2021-11-18 07:00:00     57043.55880
2021-11-18 08:00:00     54735.16896
2021-11-18 09:00:00     4932